In [114]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import requests

In [115]:
def screen_frames(html_tables, index=0):
    tables = []
    for DataFrame in html_tables:
        try:
            if 'Climate' in DataFrame[0][0]:
                tables.append(DataFrame)
        except:
            continue
    return tables[index]

In [116]:
def is_metric(DataFrame):
    #['Record high °F (°C)', 'Mean maximum °F (°C)', 'Average high °F (°C)'...], ==> °F
    t_form = DataFrame.index.tolist()[0].split()[-2]
    return t_form[1] == 'C'

In [117]:
def reset_labels(DataFrame):
    """
    set index to first column and headers to first row.
    """
    DataFrame.columns = DataFrame.loc[1] #set columns to top row
    DataFrame.drop(1, axis=0, inplace=True) #drop top now-redundant top row
    DataFrame.set_index('Month', inplace=True) #set index to column 'month' and drop now-redundant column
    del DataFrame.index.name #delete labels on index remaining from original row
    del DataFrame.columns.name

In [118]:
def get_clean_index(DataFrame, is_metric):
    updated_index = []
    for row in DataFrame.index.tolist():
        if any(DataFrame.loc[row].str.contains('\(')):
            updated_index.append(remove_imperial_value(row, is_metric))
        else:
            updated_index.append(row)
    
    for index, row in enumerate(updated_index):
        updated_index[index] = row.translate({ord('≥'):'>='}) # '≥' (ord 8805) ==> '>='
        
    return updated_index

In [119]:
def remove_imperial_value(index_key, is_metric):
    new_index = index_key.split()
    if is_metric:
        del new_index[-1]
        new_index[-1] = f'({new_index[-1]})' #change °C/mm/cm to (°C)/(mm)/(cm)
        return ' '.join(new_index)
    else:
        del new_index[-2]
        return ' '.join(new_index) #metric value already in brackets

In [120]:
def filter_imperial(city, is_metric):
    patterns = {True: r'(.*)\s\(', False: r'\((.*)\)'}
    pat = patterns[is_metric]
    for row in city.index: 
        if any(city.loc[row].str.contains('\(')):
            city.loc[row] = city.loc[row].str.extract(pat, expand=False)

In [121]:
def fix_numerical_formatting(string):
    """
    removes formatting which is incompatible with excel and float() call.
    '−' (ord 8722) ==> '-' (ord 45)
    ',' ==> '' (empty character)
    """
    try:
        fixed_value = string.translate({ord('−'):'-', ord(','):''}) 
        return fixed_value
    except AttributeError: #some cells will contain float(0)/NaN rather than strings due to 'trace' being replaced by Null by filter_imperial()
        return 0

In [122]:
def get_city_tables(city_name):
    """
    pd.read_html(HTML_address) has an issue where it can't deal with unicode characters even if the encoding is set to
    encoding='utf-8'. This means that it will throw an error if passed 'São_Paulo' for example. This function is a work around 
    which gets the page HTML via requests.get and passes the responce's text attribute to pd.read_html instead.
    """
    return pd.read_html(requests.get(f'https://en.wikipedia.org/wiki/{city_name}').text)

In [123]:
def get_climate_data(city_name, page_table_index=0):
#    city_page = pd.read_html(f'https://en.wikipedia.org/wiki/{city_name}')
    city_page = get_city_tables(city_name)
    try:
        print(f'processing: {city_name}')
        city = screen_frames(city_page, page_table_index)  #pulls out the first climate table from the page HTML
        city.dropna(inplace=True) #drops all rows with null values (i.e. descriptions)
        reset_labels(city) #pull real headers and index labels out of dataframe
        is_data_metric = is_metric(city)
        city.set_index([get_clean_index(city, is_data_metric)], inplace=True) #removes imperial values from index 
        filter_imperial(city, is_data_metric) #removes all imperial secondary values in city.values
        city = city.applymap(fix_numerical_formatting) #removes commas and fixes unicode minus signs which interfere with float()
        city = city.applymap(float) #converts all values in the table from str to float 
        return city

    except IndexError:  # screen_frames will throw an IndexError if there are no climate data tables in the wiki-page html
        print(f'No climate data for {city_name}')

In [124]:
cities = ['New_York_City', 'Shanghai', 'Tokyo', 'Chongqing', 'San_Francisco', 'Los_Angeles', 'Seattle', 'Manchester', 'London', 'Osaka', 'San_Diego', 'Chengdu', 'Kobe', 'Lisbon', 'Shenzhen', 'Sydney', 'Melbourne', 'Gold_Coast,_Queensland', 'Cape_Town', 'Nairobi', 'Hong_Kong', 'Washington,_D.C.', 'Frankfurt', 'Philadelphia', 'Rio_de_Janeiro', 'Houston', 'Austin', 'Kyoto', 'Dallas', 'Joshua_Tree,_California', 'Toronto', 'Detroit', 'Nagoya', 'Boston', 'Vancouver', 'Paris', 'Berlin', 'Copenhagen', 'Barcelona', 'Rome', 'Seoul', 'São_Paulo', 'Reykjavík']

In [125]:
city_data = {city:get_climate_data(city) for city in cities} #generate dictionary {'city_name': DataFrame}
city_data['Chicago'] = get_climate_data('Chicago', 1) #some cities have multiple climate data tables with varying amounts of data. I wanted the ones with sunshine data from Chicago and Dublin
city_data['Dublin'] = get_climate_data('Dublin', 1)

processing: New_York_City
processing: Shanghai
processing: Tokyo
processing: Chongqing
processing: San_Francisco
processing: Los_Angeles
processing: Seattle
processing: Manchester
processing: London
processing: Osaka
processing: San_Diego
processing: Chengdu
processing: Kobe
processing: Lisbon
processing: Shenzhen
processing: Sydney
processing: Melbourne
processing: Gold_Coast,_Queensland
processing: Cape_Town
processing: Nairobi
processing: Hong_Kong
processing: Washington,_D.C.
processing: Frankfurt
processing: Philadelphia
processing: Rio_de_Janeiro
processing: Houston
processing: Austin
processing: Kyoto
processing: Dallas
processing: Joshua_Tree,_California
processing: Toronto
processing: Detroit
processing: Nagoya
processing: Boston
processing: Vancouver
processing: Paris
processing: Berlin
processing: Copenhagen
processing: Barcelona
processing: Rome
processing: Seoul
processing: São_Paulo
processing: Reykjavík
processing: Chicago
processing: Dublin


In [131]:
climate_data = pd.concat(city_data) #concatenate city_data into a MultiIndex df

In [133]:
climate_data.to_csv('city_climate_data_newest.csv', encoding='utf-8')

In [134]:
data = pd.read_csv('city_climate_data_newest.csv', encoding='utf-8', index_col=[0, 1])